In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/startup-failures/Startup Failure (Finance and Insurance).csv
/kaggle/input/startup-failures/Startup Failures.csv
/kaggle/input/startup-failures/Startup Failure (Manufactures).csv
/kaggle/input/startup-failures/Startup Failure (Retail Trade).csv
/kaggle/input/startup-failures/Startup Failure (Health Care).csv
/kaggle/input/startup-failures/Startup Failure (Food and services).csv
/kaggle/input/startup-failures/Startup Failures (Information Sector).csv


In [39]:
data = pd.read_csv("/kaggle/input/startup-failures/Startup Failures.csv")
data_FandI = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Finance and Insurance).csv")
data_Manu = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Manufactures).csv")
data_Retail = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Retail Trade).csv")
data_HealthC = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Health Care).csv")
data_Food = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Food and services).csv")
data_Info = pd.read_csv("/kaggle/input/startup-failures/Startup Failures (Information Sector).csv")

datas = [data,data_FandI, data_Manu,data_Retail,data_HealthC, data_Food,data_Info]
datas_n = ["data","data_FandI", "data_Manu","data_Retail","data_HealthC", "data_Food","data_Info"]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Name                815 non-null    object
 1   Sector              815 non-null    object
 2   Years of Operation  815 non-null    object
dtypes: object(3)
memory usage: 19.2+ KB


In [44]:
# Process to Merge the Data

i=0
for data in datas:
    print("\n")
    print(datas_n[i])
    print("\n")
    i+=1
    data.info()
    



data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Name                815 non-null    object
 1   Sector              815 non-null    object
 2   Years of Operation  815 non-null    object
dtypes: object(3)
memory usage: 19.2+ KB


data_FandI


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    47 non-null     object 
 1   Sector                  47 non-null     object 
 2   Years of Operation      47 non-null     object 
 3   What They Did           47 non-null     object 
 4   How Much They Raised    47 non-null     object 
 5   Why They Failed         47 non-null     object 
 6   Takeaway                47 non-null     object 
 7   Giants

In [68]:
# Merging Data

dfs = [data_FandI, data_Manu, data_Retail, data_HealthC, data_Food, data_Info]
merged_data = pd.concat(dfs, ignore_index=True, sort=False)

print(merged_data.shape)

(409, 21)


In [65]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    409 non-null    object 
 1   Sector                  409 non-null    object 
 2   Years of Operation      409 non-null    object 
 3   What They Did           409 non-null    object 
 4   How Much They Raised    409 non-null    object 
 5   Why They Failed         409 non-null    object 
 6   Takeaway                409 non-null    object 
 7   Giants                  409 non-null    int64  
 8   No Budget               409 non-null    int64  
 9   Competition             409 non-null    int64  
 10  Poor Market Fit         409 non-null    int64  
 11  Acquisition Stagnation  409 non-null    int64  
 12  Platform Dependency     383 non-null    float64
 13  Monetization Failure    409 non-null    int64  
 14  Niche Limits            409 non-null    in

In [66]:
data_copy = merged_data.copy()

In [67]:
merged_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Name,Sector,Years of Operation,What They Did,How Much They Raised,Why They Failed,Takeaway,Giants,No Budget,Competition,...,Acquisition Stagnation,Platform Dependency,Monetization Failure,Niche Limits,Execution Flaws,Trend Shifts,Toxicity/Trust Issues,Regulatory Pressure,Overhype,High Operational Costs
0,Avant,Finance and Insurance,2012-2023,Online personal loans,$655M,Lost to LendingClub and high defaults,Lending needs risk balance,1,1,1,...,0,0.0,0,0,0,0,0.0,0.0,0.0,NaN
1,Bitpass,Finance and Insurance,2002-2008,Micropayments platform,$2M,Lost to PayPal and low adoption,Micropayments need mass use,1,0,1,...,0,0.0,0,0,0,0,0.0,0.0,0.0,NaN
2,Cake Financial,Finance and Insurance,2006-2011,Portfolio tracking tool,$3M,Lost to Mint and sold to TradeKing,Finance tools need scale,1,0,1,...,1,0.0,0,0,0,0,0.0,0.0,0.0,NaN
3,Circle,Finance and Insurance,2013-2023,Crypto payments and stablecoin,$500M,Lost to Coinbase and market shifts,Crypto needs stability,1,0,1,...,0,0.0,0,0,0,1,0.0,0.0,0.0,NaN
4,Clarity Money,Finance and Insurance,2016-2022,Personal finance app,$11M,Lost to Mint/Acorns and sold to Goldman,Finance apps need edge,1,0,1,...,1,0.0,0,0,0,0,0.0,0.0,0.0,NaN


In [81]:
# Converting the Funding amounts from string/ object type to float type

merged_data[["Amount","Unit"]] = merged_data['How Much They Raised'].str.extract(r'(\d+\.?\d*)([MBK]?)')
merged_data["Amount"] = pd.to_numeric(merged_data['Amount'], errors = "coerce")
multipliers = {"M":1, "B":1000, "K":1/1000}
merged_data['Funding Amount'] = merged_data["Amount"]*merged_data["Unit"].map(multipliers).fillna(1)
merged_data['Funding Amount'] = merged_data['Funding Amount'].astype("Float64")
merged_data.drop(columns=["Amount", "Unit"], inplace = True)

merged_data.tail()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Name,Sector,Years of Operation,What They Did,How Much They Raised,Why They Failed,Takeaway,Giants,No Budget,Competition,...,Platform Dependency,Monetization Failure,Niche Limits,Execution Flaws,Trend Shifts,Toxicity/Trust Issues,Regulatory Pressure,Overhype,High Operational Costs,Funding Amount
404,GoAnimate,Information,14 (2007-2021),DIY animation tool,$10M,Sold 2021; lost to Powtoon,Scale beats tools,1,0,1,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,10.0
405,Hitpost,Information,4 (2009-2013),Sports social app,$1M,Closed 2013; lost to X,Giants own fandom,1,0,0,...,0.0,0,1,0,0,0.0,0.0,0.0,NaN,1.0
406,Homer,Information,6 (2013-2019),Kids' reading app,$2.2M,Sold 2019; lost to Epic!,Kids' ed needs scale,1,0,1,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,2.2
407,Hot Potato,Information,2 (2009-2011),Social check-in app,$1.4M,Acquired 2011; shut down; lost to Foursquare,Early exits end,0,0,1,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,1.4
408,Humanoid,Information,5 (2012-2017),Content curation platform,$1M,Closed 2017; lost to Medium,Curation needs clout,1,0,1,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,1.0


In [87]:
billionRows = merged_data[merged_data["How Much They Raised"].str.contains("B",na=False)]
billionRows

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Name,Sector,Years of Operation,What They Did,How Much They Raised,Why They Failed,Takeaway,Giants,No Budget,Competition,...,Platform Dependency,Monetization Failure,Niche Limits,Execution Flaws,Trend Shifts,Toxicity/Trust Issues,Regulatory Pressure,Overhype,High Operational Costs,Funding Amount
12,LendingClub,Finance and Insurance,2006-2021,P2P lending platform,$1B,Lost to banks and trust issues,P2P needs trust,1,0,1,...,0.0,0,0,0,0,1.0,0.0,0.0,NaN,1000.0
56,Dyson's EV Project,Manufacturing,2017-2019,Electric vehicle,$2.7B,Lost to Tesla/Nio and high costs,EVs are a giant’s game,1,1,1,...,0.0,0,0,0,0,0.0,0.0,1.0,NaN,2700.0
59,Faraday Future,Manufacturing,2014-2023,Luxury electric vehicles,$3.5B,Mismanagement and Tesla/Rivian competition,EV hype needs execution,1,1,1,...,0.0,0,0,1,0,0.0,0.0,1.0,NaN,3500.0
60,Fisker Automotive,Manufacturing,2007-2013,Luxury hybrid-electric cars,$1.4B,Lost to Tesla and supplier failure,Luxury EVs need reliability,1,1,1,...,1.0,0,0,1,0,0.0,0.0,0.0,NaN,1400.0
89,ContextLogic,Retail Trade,13 (2010-2023),Discount e-commerce app (Wish),$1.8B,Faded 2023; low-quality goods; lost to Amazon,Cheap doesn't last,1,0,1,...,0.0,0,0,0,1,1.0,0.0,0.0,NaN,1800.0
163,Zappos Labs,Retail Trade,5 (2013-2018),Experimental retail projects,$1.5B (Zappos),Faded 2018; lost to Amazon core; low impact,Experiments need wins,1,0,0,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,1500.0
164,Zappos Local,Retail Trade,5 (2015-2020),Local shopping platform,$1.5B (Zappos),Faded 2020; lost to Amazon; low uptake,Local needs pull,1,0,0,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,1500.0
223,Theranos,Health Care,2003-2018,Blood testing tech,$1.4B,Fraud and tech didn’t work,Transparency is key,0,0,0,...,0.0,0,0,0,0,1.0,0.0,1.0,NaN,1400.0
270,Dropbox Paper,Information,6 (2015-2021),Collaborative doc tool,$1.7B (Dropbox),Shut down 2021; lost to Google Docs; Notion,Stick to strengths,1,0,1,...,0.0,0,0,0,0,0.0,0.0,0.0,NaN,1700.0
278,GeoCities,Information,11 (1994-2005),Web hosting for personal pages,$0 (Yahoo $3.6B),Shut down 2005; Yahoo neglect; lost to blogs,Adapt or atrophy,1,0,0,...,0.0,0,0,0,1,0.0,0.0,0.0,NaN,0.0


In [89]:
print(merged_data["Funding Amount"].isna().sum())  # Count NaN values
merged_data = merged_data.dropna(subset=["Funding Amount"])


1


In [90]:
print(merged_data["Funding Amount"].isna().sum())

0


In [92]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408 entries, 0 to 408
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    408 non-null    object 
 1   Sector                  408 non-null    object 
 2   Years of Operation      408 non-null    object 
 3   What They Did           408 non-null    object 
 4   How Much They Raised    408 non-null    object 
 5   Why They Failed         408 non-null    object 
 6   Takeaway                408 non-null    object 
 7   Giants                  408 non-null    int64  
 8   No Budget               408 non-null    int64  
 9   Competition             408 non-null    int64  
 10  Poor Market Fit         408 non-null    int64  
 11  Acquisition Stagnation  408 non-null    int64  
 12  Platform Dependency     382 non-null    float64
 13  Monetization Failure    408 non-null    int64  
 14  Niche Limits            408 non-null    int64  

In [94]:
# Making a column to capture the years of Operation in Float type

merged_data['Years of Operation'].unique()

array(['2012-2023', '2002-2008', '2006-2011', '2013-2023', '2016-2022',
       '2011-2016', '2021-2023', '2012-2022', '2010-2016', '2014-2019',
       '2012-2020', '2012-2021', '2006-2021', '2013-2017', '2008-2018',
       '2005-2008', '2008-2017', '2011-2017', '2002-2007', '2015-2020',
       '2009-2013', '2010-2017', '2003-2018', '2012-2017', '2016-2020',
       '2007-2018', '2009-2016', '2016-2021', '2013-2020', '2009-2022',
       '2012-2018', '2008-2020', '2012-2016', '2015-2021', '2008-2015',
       '2018-2023', '2017-2023', '2011-2023', '2009-2021', '2009-2020',
       '2011-2018', '2010-2019', '2005-2011', '2008-2019', '2007-2013',
       '2011-2020', '2017-2019', '2014-2023', '2015-2018', '2006-2017',
       '2006-2018', '2009-2017', '2014-2017', '2013-2016', '2015-2023',
       '3 (2010-2013)', '7 (2010-2017)', '10 (2006-2016)',
       '6 (2012-2018)', '5 (2012-2017)', '4 (2013-2017)', '8 (2013-2021)',
       '7 (2015-2022)', '6 (2009-2015)', '3 (2015-2018)', '8 (2012-2020)',

In [106]:
# Extracting the integer value of the number of years of the operations of the company ane making a new column out of it.

pattern = r'(?:(\d+) \()?(\d{4})-(\d{4})\)?'

# Extract years
merged_data[["Years_Extracted", "Start_Year", "End_Year"]] = merged_data["Years of Operation"].str.extract(pattern)

# Convert Start_Year and End_Year to integers safely
merged_data["Start_Year"] = pd.to_numeric(merged_data["Start_Year"], errors="coerce").astype("Int64")
merged_data["End_Year"] = pd.to_numeric(merged_data["End_Year"], errors="coerce").astype("Int64")
merged_data["Years_Extracted"] = pd.to_numeric(merged_data["Years_Extracted"], errors="coerce")  # Keep as float

# Compute years of operation
merged_data["Years of Operation (Int)"] = merged_data["Years_Extracted"].fillna(
    merged_data["End_Year"].sub(merged_data["Start_Year"])
).astype("Int64")

# Drop intermediate columns
merged_data.drop(columns=["Years_Extracted", "Start_Year", "End_Year"], inplace=True)

# Check final result
merged_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Name,Sector,Years of Operation,What They Did,How Much They Raised,Why They Failed,Takeaway,Giants,No Budget,Competition,...,Niche Limits,Execution Flaws,Trend Shifts,Toxicity/Trust Issues,Regulatory Pressure,Overhype,High Operational Costs,Funding Amount,End_year,Years of Operation (Int)
0,Avant,Finance and Insurance,2012-2023,Online personal loans,$655M,Lost to LendingClub and high defaults,Lending needs risk balance,1,1,1,...,0,0,0,0.0,0.0,0.0,NaN,655.0,NaN,11
1,Bitpass,Finance and Insurance,2002-2008,Micropayments platform,$2M,Lost to PayPal and low adoption,Micropayments need mass use,1,0,1,...,0,0,0,0.0,0.0,0.0,NaN,2.0,NaN,6
2,Cake Financial,Finance and Insurance,2006-2011,Portfolio tracking tool,$3M,Lost to Mint and sold to TradeKing,Finance tools need scale,1,0,1,...,0,0,0,0.0,0.0,0.0,NaN,3.0,NaN,5
3,Circle,Finance and Insurance,2013-2023,Crypto payments and stablecoin,$500M,Lost to Coinbase and market shifts,Crypto needs stability,1,0,1,...,0,0,1,0.0,0.0,0.0,NaN,500.0,NaN,10
4,Clarity Money,Finance and Insurance,2016-2022,Personal finance app,$11M,Lost to Mint/Acorns and sold to Goldman,Finance apps need edge,1,0,1,...,0,0,0,0.0,0.0,0.0,NaN,11.0,NaN,6


In [108]:
# check for NaN values in the new column
merged_data["Years of Operation (Int)"].isna().sum()

0

In [115]:
# drop the unnecessary columns
merged_data.drop(columns=['Years of Operation', 'How Much They Raised'], inplace=True)

In [114]:
merged_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Name,Sector,What They Did,Why They Failed,Takeaway,Giants,No Budget,Competition,Poor Market Fit,Acquisition Stagnation,...,Niche Limits,Execution Flaws,Trend Shifts,Toxicity/Trust Issues,Regulatory Pressure,Overhype,High Operational Costs,Funding Amount,End_year,Years of Operation (Int)
0,Avant,Finance and Insurance,Online personal loans,Lost to LendingClub and high defaults,Lending needs risk balance,1,1,1,0,0,...,0,0,0,0.0,0.0,0.0,NaN,655.0,NaN,11
1,Bitpass,Finance and Insurance,Micropayments platform,Lost to PayPal and low adoption,Micropayments need mass use,1,0,1,1,0,...,0,0,0,0.0,0.0,0.0,NaN,2.0,NaN,6
2,Cake Financial,Finance and Insurance,Portfolio tracking tool,Lost to Mint and sold to TradeKing,Finance tools need scale,1,0,1,0,1,...,0,0,0,0.0,0.0,0.0,NaN,3.0,NaN,5
3,Circle,Finance and Insurance,Crypto payments and stablecoin,Lost to Coinbase and market shifts,Crypto needs stability,1,0,1,0,0,...,0,0,1,0.0,0.0,0.0,NaN,500.0,NaN,10
4,Clarity Money,Finance and Insurance,Personal finance app,Lost to Mint/Acorns and sold to Goldman,Finance apps need edge,1,0,1,0,1,...,0,0,0,0.0,0.0,0.0,NaN,11.0,NaN,6


In [119]:
# check for unique values of sector
merged_data["Sector"].value_counts()

Sector
Information                        156
Retail Trade                        90
Health Care                         59
Finance and Insurance               47
Manufacturing                       30
Accommodation and Food Services     26
Name: count, dtype: int64